In [1]:
import tensorflow as tf
import openl3
import librosa
import numpy as np
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import silhouette_score
import os
import random

# gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)
print("gpu:", gpus)

gpu: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
random.seed(42)

paths = {
    "drones": "../data/drones",
    "not_drones": "../data/not_drones"
}
SAVE_PATH = "../embeddings/openl3_embeddings.npz"

def collect_files(path):
    fs = []
    for root, _, files in os.walk(path):
        for f in files:
            if f.lower().endswith((".wav", ".mp3")):
                fs.append(os.path.join(root, f))
    return sorted(fs)  # важно — всегда одна сортировка

In [3]:
# собираем
drones_all = collect_files(paths["drones"])
not_drones_all = collect_files(paths["not_drones"])

# берём 50%
drones_sel = drones_all[: len(drones_all) // 50]
not_drones_sel = not_drones_all[: len(not_drones_all) // 50]

# общие списки
files = drones_sel + not_drones_sel
labels = [1] * len(drones_sel) + [0] * len(not_drones_sel)

print("дронов:", len(drones_sel))
print("не дронов:", len(not_drones_sel))
print("всего:", len(files))

дронов: 1411
не дронов: 1304
всего: 2715


In [4]:
# модель
print("модель...")
model = openl3.models.load_audio_embedding_model(
    input_repr="mel256",
    content_type="env",
    embedding_size=512
)

модель...


In [5]:
# загрузка аудио
def load_audio(path):
    if not os.path.exists(path):
        return None  # файл пропущен
    try:
        audio, sr = librosa.load(path, sr=None, mono=True)
        return audio, sr
    except Exception:
        return None


print("загрузка...")
loaded = []
with ThreadPoolExecutor(max_workers=8) as ex:
    for item in tqdm(ex.map(load_audio, files), total=len(files), desc="аудио"):
        if item is not None:
            loaded.append(item)

загрузка...


аудио: 100%|██████████| 2715/2715 [00:03<00:00, 852.64it/s] 


In [6]:
print("эмбеддинги...")
embeddings = []
batch_size = 16
processed = 0

for i in range(0, len(loaded), batch_size):
    batch = loaded[i:i+batch_size]
    if len(batch) == 0:
        continue

    audios = [a for (a, sr) in batch]
    srs = [sr for (a, sr) in batch]

    target_len = max(len(a) for a in audios)
    audios = [librosa.util.fix_length(a, size=target_len) for a in audios]

    for audio in audios:
        try:
            # получаем эмбеддинг строго по одному файлу
            e, _ = openl3.get_audio_embedding(
                audio,
                srs[0],
                embedding_size=512,
                content_type="env",
                model=model,
                verbose=False  # нет вывода tf
            )

            # mean pooling по окнам
            e = np.mean(e, axis=0)

            # проверяем что размер ровно 512
            if e.shape != (512,):
                continue

            embeddings.append(e)
            processed += 1

            if processed % 100 == 0:
                print(f"обработано: {processed}")

        except Exception:
            continue

эмбеддинги...
обработано: 100
обработано: 200
обработано: 300
обработано: 400
обработано: 500
обработано: 600
обработано: 700
обработано: 800
обработано: 900
обработано: 1000
обработано: 1100
обработано: 1200
обработано: 1300
обработано: 1400
обработано: 1500
обработано: 1600
обработано: 1700
обработано: 1800
обработано: 1900
обработано: 2000
обработано: 2100
обработано: 2200
обработано: 2300
обработано: 2400
обработано: 2500
обработано: 2600
обработано: 2700


In [ ]:
print("эмбеддингов:", len(embeddings))

if len(embeddings) == 0:
    print("нет эмбеддингов — проверь данные")
else:
    X = np.vstack(embeddings)
    y = np.array(labels[: len(X)])
    print("X форма:", X.shape)
    print("y форма:", y.shape)

эмбеддингов: 2715
X форма: (2715, 512)
y форма: (2715,)


In [9]:
# метрика
score = silhouette_score(X, y)
print("sil:", score)

# путь сохранения
save_dir = "../embeddings"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "openl3.npz")

# сохранение
np.savez(save_path, X=X, y=y, score=score)
print("сохранено:", save_path)


sil: 0.4626370072364807
сохранено: ../embeddings\openl3.npz
